In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score

/Users/beyzakaya/miniforge3/envs/DLprojectenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd 
from datasets import load_dataset, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
def prepare_datasets(tokenizer, dataset_name="financial_phrasebank", subset_name="sentences_50agree", max_length=128, random_state=42):
    dataset = load_dataset(dataset_name, subset_name, trust_remote_code=True)
    
    df = pd.DataFrame(dataset['train'])

    # Stratify split into train, validation, and test
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        df['sentence'], df['label'], test_size=0.2, stratify=df['label'], random_state=random_state
    )
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_texts, train_labels, test_size=0.1, stratify=train_labels, random_state=random_state
    )

    # Create DataFrames for each split
    train_df = pd.DataFrame({'sentence': train_texts, 'label': train_labels})
    val_df = pd.DataFrame({'sentence': val_texts, 'label': val_labels})
    test_df = pd.DataFrame({'sentence': test_texts, 'label': test_labels})

    # Convert DataFrames to Hugging Face Dataset format
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Define tokenization function
    def tokenize_function(example):
        return tokenizer(
            example["sentence"], 
            padding="max_length", 
            truncation=True, 
            max_length=max_length
        )
     # Tokenize datasets
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Remove raw text and prepare for Hugging Face Trainer
    train_dataset = train_dataset.remove_columns(["sentence"])
    val_dataset = val_dataset.remove_columns(["sentence"])
    test_dataset = test_dataset.remove_columns(["sentence"])

    train_dataset = train_dataset.rename_column("label", "labels")
    val_dataset = val_dataset.rename_column("label", "labels")
    test_dataset = test_dataset.rename_column("label", "labels")

    train_dataset.set_format("torch")
    val_dataset.set_format("torch")
    test_dataset.set_format("torch")

    return train_dataset, val_dataset, test_dataset

In [13]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the tokenizer for BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the Teacher model (BERT)
teacher_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Load the Student model (DistilBERT)
student_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
train_dataset, val_dataset, test_dataset = prepare_datasets(tokenizer)

Map: 100%|██████████| 970/970 [00:00<00:00, 22919.05 examples/s]


In [31]:
import torch
import torch.nn.functional as F

def distillation_loss(student_logits, teacher_logits, true_labels, temperature=2.0, alpha=0.7):
    # Soft labels from teacher model
    soft_labels = F.softmax(teacher_logits / temperature, dim=-1)

    # Hard loss using true labels
    hard_loss = F.cross_entropy(student_logits, true_labels)

    # Soft loss between teacher and student logits
    soft_loss = F.kl_div(
        F.log_softmax(student_logits / temperature, dim=-1),
        soft_labels,
        reduction='batchmean'
    )

    # Weighted combination of soft and hard loss
    return alpha * soft_loss + (1.0 - alpha) * hard_loss



In [32]:
from torch.utils.data import DataLoader

def get_teacher_logits(model, dataset, batch_size=16, device='cpu'):
    dataloader = DataLoader(dataset, batch_size=batch_size)
    logits = []

    model.eval()  # Set teacher model to evaluation mode
    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
            outputs = model(**inputs)
            logits.append(outputs.logits.cpu())

    return torch.cat(logits, dim=0)  # Concatenate all logits into one tensor


In [35]:
def distillation_train_loop(student_model, teacher_model, train_dataset, val_dataset, tokenizer, epochs=3, batch_size=16, learning_rate=5e-5, device='cpu'):
    student_model = student_model.to(device)
    teacher_model = teacher_model.to(device)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

    optimizer = torch.optim.AdamW(student_model.parameters(), lr=learning_rate)

    # Teacher logits for the entire training dataset
    teacher_logits = get_teacher_logits(teacher_model, train_dataset, batch_size=batch_size, device=device)
    print(f"Teacher logits: {teacher_logits}")

    for epoch in range(epochs):
        student_model.train()
        total_loss = 0.0

        for i, batch in enumerate(train_dataloader):
            inputs = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
            labels = batch['labels'].to(device)

            # Select corresponding teacher logits for this batch
            batch_teacher_logits = teacher_logits[i * batch_size:(i + 1) * batch_size].to(device)

            # Forward pass of student model
            student_outputs = student_model(**inputs)
            print(f"Student outputs: {student_outputs}")
            student_logits = student_outputs.logits
            print(f"Student logits: {student_logits}")

            # Compute loss
            loss = distillation_loss(student_logits, batch_teacher_logits, labels)
            total_loss += loss.item()

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_dataloader):.4f}")

    return student_model



In [23]:
# Set up the training arguments for the teacher model
training_args = TrainingArguments(
    output_dir="./results_knowledge_distill/teacher_model",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs_knowledge_distill/teacher_model",
    logging_steps=10,
)

# Trainer for the teacher model
trainer_teacher = Trainer(
    model=teacher_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Fine-tune the teacher model
trainer_teacher.train()


/Users/beyzakaya/miniforge3/envs/DLprojectenv/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/kv/563b5k8n4xg24_t9kd2d72c40000gn/T/ipykernel_56174/2655727639.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_teacher = Trainer(
  2%|▏         | 10/654 [00:22<15:48,  1.47s/it] 

{'loss': 0.8957, 'grad_norm': 4.335817337036133, 'learning_rate': 4.923547400611621e-05, 'epoch': 0.05}


  3%|▎         | 20/654 [00:35<11:03,  1.05s/it]

{'loss': 0.7797, 'grad_norm': 3.726905584335327, 'learning_rate': 4.847094801223242e-05, 'epoch': 0.09}


  5%|▍         | 30/654 [00:55<19:18,  1.86s/it]

...

 99%|█████████▉| 650/654 [16:58<00:04,  1.02s/it]

{'loss': 0.1114, 'grad_norm': 0.08660129457712173, 'learning_rate': 3.0581039755351683e-07, 'epoch': 2.98}


                                                 
100%|██████████| 654/654 [17:20<00:00,  1.59s/it]

{'eval_loss': 0.6311530470848083, 'eval_runtime': 7.7797, 'eval_samples_per_second': 49.873, 'eval_steps_per_second': 3.213, 'epoch': 3.0}
{'train_runtime': 1040.5886, 'train_samples_per_second': 10.056, 'train_steps_per_second': 0.628, 'train_loss': 0.2857098263611487, 'epoch': 3.0}


TrainOutput(global_step=654, training_loss=0.2857098263611487, metrics={'train_runtime': 1040.5886, 'train_samples_per_second': 10.056, 'train_steps_per_second': 0.628, 'total_flos': 688304700776448.0, 'train_loss': 0.2857098263611487, 'epoch': 3.0})

#### This function ensures the student model is trained using the logits from the teacher model in addition to the ground truth labels, combining hard and soft losses.

In [36]:
# Now, distill the knowledge into the student model
student_model = distillation_train_loop(student_model, teacher_model, train_dataset, val_dataset, tokenizer, epochs=3, batch_size=16)

Teacher logits: tensor([[-2.6213,  4.1635, -2.3194],
        [-1.5190, -1.7753,  4.8557],
        [-2.5256,  4.1303, -2.2848],
        ...,
        [-1.7173,  3.6361, -2.9695],
        [-1.8634, -1.4155,  4.7208],
        [-2.6480,  4.1266, -2.2814]])
Student outputs: SequenceClassifierOutput(loss=None, logits=tensor([[-1.4568,  1.6938, -0.6216],
        [-1.4918,  1.5309, -0.0303],
        [-1.6417,  1.1014,  0.4621],
        [-1.2778,  1.1750, -0.1223],
        [-1.7693,  0.3149,  1.4295],
        [-1.8186,  1.1852,  0.1105],
        [ 0.1959,  0.1122, -0.8044],
        [-2.2644,  0.2283,  1.6057],
        [-1.8654,  0.4816,  0.9782],
        [-0.0396,  0.2007, -0.8431],
        [-1.1631,  1.3751, -0.3029],
        [-1.7715,  1.5458,  0.2369],
        [-1.1664,  1.3201, -0.6355],
        [-1.6168, -0.0067,  1.1921],
        [-1.7620,  0.3772,  1.1147],
        [-1.8191,  1.2931,  0.3181]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
Student logits: tensor([[-1.456

#### After the student model is trained, HuggingFace Trainer is used for evaluation. evaluate() method in Trainer does not perform training again, it only uses the student model in evaluation mode

In [37]:
trainer_student = Trainer(
    model=student_model,
    args=training_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# Evaluate the student model
results = trainer_student.evaluate()
print(f"Evaluation Results: {results}")


/var/folders/kv/563b5k8n4xg24_t9kd2d72c40000gn/T/ipykernel_56174/4119820128.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_student = Trainer(
100%|██████████| 61/61 [00:07<00:00,  7.97it/s]

Evaluation Results: {'eval_loss': 0.5431520938873291, 'eval_model_preparation_time': 0.0042, 'eval_runtime': 8.9747, 'eval_samples_per_second': 108.081, 'eval_steps_per_second': 6.797}


In [38]:
predictions = trainer_student.predict(test_dataset)

100%|██████████| 61/61 [00:07<00:00,  8.04it/s]


In [39]:
import numpy as np

predicted_labels = np.argmax(predictions.predictions, axis=-1)

true_labels = predictions.label_ids

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8062
Precision: 0.8145
Recall: 0.8062
F1 Score: 0.7975
